In [69]:
import io
import pandas as pd
import numpy as np

pd.options.display.max_colwidth = 20
pd.options.display.max_rows = 40

In [70]:
%%time
file_path = './data/processed/dataset.pkl.zst'
DATASET = pd.read_pickle(file_path)
DATASET

CPU times: user 87.9 ms, sys: 36 ms, total: 124 ms
Wall time: 123 ms


,CD_CVM,CNPJ_CIA,DENOM_CIA,is_annual,is_consolidated,fs_type,DT_REFER,VERSAO,DT_INI_EXERC,DT_FIM_EXERC,ORDEM_EXERC,CD_CONTA,DS_CONTA,ST_CONTA_FIXA,COLUNA_DF,VL_CONTA
0,94,92.693.019/0001-89,PANATLANTICA S.A.,True,True,1,2010-12-31,2,NaT,2009-12-31,-1,1,Ativo Total,True,NaN,211293000.0
1,94,92.693.019/0001-89,PANATLANTICA S.A.,True,False,1,2010-12-31,2,NaT,2009-12-31,-1,1,Ativo Total,True,NaN,191541000.0
2,94,92.693.019/0001-89,PANATLANTICA S.A.,True,True,1,2010-12-31,2,NaT,2009-12-31,-1,1.01,Ativo Circulante,True,NaN,164307000.0
3,94,92.693.019/0001-89,PANATLANTICA S.A.,True,False,1,2010-12-31,2,NaT,2009-12-31,-1,1.01,Ativo Circulante,True,NaN,127423000.0
4,94,92.693.019/0001-89,PANATLANTICA S.A.,True,True,1,2010-12-31,2,NaT,2009-12-31,-1,1.01.01,Caixa e Equivale...,True,NaN,2055000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6783055,518700,70.216.429/0001-04,BRAMEX - BRASIL ...,True,False,7,2013-12-31,1,2013-01-01,2013-12-31,0,7.08.03,Remuneração de C...,True,NaN,328371.0
6783056,518700,70.216.429/0001-04,BRAMEX - BRASIL ...,True,False,7,2013-12-31,1,2013-01-01,2013-12-31,0,7.08.03.01,Juros,True,NaN,328371.0
6783057,518700,70.216.429/0001-04,BRAMEX - BRASIL ...,True,False,7,2013-12-31,1,2013-01-01,2013-12-31,0,7.08.04,Remuneração de C...,True,NaN,-1200611.0
6783058,518700,70.216.429/0001-04,BRAMEX - BRASIL ...,True,False,7,2013-12-31,1,2013-01-01,2013-12-31,0,7.08.04.03,Lucros Retidos /...,True,NaN,-1200611.0


In [71]:
DATASET.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6783060 entries, 0 to 6783059
Data columns (total 16 columns):
 #   Column           Dtype   
---  ------           -----   
 0   CD_CVM           category
 1   CNPJ_CIA         category
 2   DENOM_CIA        category
 3   is_annual        category
 4   is_consolidated  category
 5   fs_type          category
 6   DT_REFER         category
 7   VERSAO           category
 8   DT_INI_EXERC     category
 9   DT_FIM_EXERC     category
 10  ORDEM_EXERC      category
 11  CD_CONTA         category
 12  DS_CONTA         category
 13  ST_CONTA_FIXA    category
 14  COLUNA_DF        category
 15  VL_CONTA         category
dtypes: category(16)
memory usage: 217.3 MB


In [62]:
# df_dataset.query("CD_CVM == 9512")
# df_dataset.query("VL_CONTA == 0")
DATASET.VL_CONTA.nunique()


987144

In [76]:
class Company():

    def __init__(self, cvm_number: int, consolidated=True):
        self.cvm_number = int(cvm_number)
        self.consolidated = consolidated
        self._df = DATASET.query(
            "(CD_CVM == @self.cvm_number) and \
             (is_consolidated == @self.consolidated)"
        ).copy()
        self.remove_category()

    def remove_category(self):
        self._df = self._df.astype({
            'CD_CVM': 'object',
            'is_annual': bool,
            'fs_type': np.int8,
            'is_consolidated': bool,
            'DT_REFER': 'datetime64',
            'DT_INI_EXERC': 'datetime64',
            'DT_FIM_EXERC': 'datetime64',
            'ORDEM_EXERC': np.int8,
            'CD_CONTA': 'object',
            'DS_CONTA': 'object',
            'ST_CONTA_FIXA': bool,
            'COLUNA_DF': 'object',
            'VL_CONTA': float
        })
        return

    def assets(self) -> pd.DataFrame:
        df = self._df.query("fs_type == 1").copy()
        columns_drop = ['CNPJ_CIA', 'DENOM_CIA', 'VERSAO', 'ORDEM_EXERC']
        df.drop(columns=columns_drop, inplace=True)
        # df = self.remove_category()
        df.query("CD_CONTA == '1'", inplace=True)
        # sort for drop operation
        df.sort_values(['DT_FIM_EXERC', 'DT_REFER', 'CD_CONTA'], inplace=True)
        # only the last published statement will be used
        df.drop_duplicates(
            subset=['DT_FIM_EXERC', 'CD_CONTA'],
            keep='last',
            inplace=True,
            ignore_index=True)
        df.reset_index(drop=True, inplace=True)
        return df


In [77]:
%%time
petro = Company(9512)
df = petro.assets()
df

CPU times: user 25.1 ms, sys: 8.38 ms, total: 33.5 ms
Wall time: 32.8 ms


,CD_CVM,is_annual,is_consolidated,fs_type,DT_REFER,DT_INI_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,ST_CONTA_FIXA,COLUNA_DF,VL_CONTA
0,9512,True,True,1,2010-12-31,NaT,2009-12-31,1,Ativo Total,True,NaN,3.504189e+11
1,9512,True,True,1,2011-12-31,NaT,2010-12-31,1,Ativo Total,True,NaN,5.168457e+11
2,9512,True,True,1,2012-12-31,NaT,2011-12-31,1,Ativo Total,True,NaN,6.000970e+11
3,9512,True,True,1,2013-12-31,NaT,2012-12-31,1,Ativo Total,True,NaN,6.690316e+11
4,9512,True,True,1,2014-12-31,NaT,2013-12-31,1,Ativo Total,True,NaN,7.529670e+11
5,9512,True,True,1,2015-12-31,NaT,2014-12-31,1,Ativo Total,True,NaN,7.933750e+11
6,9512,True,True,1,2016-12-31,NaT,2015-12-31,1,Ativo Total,True,NaN,9.001350e+11
7,9512,True,True,1,2017-12-31,NaT,2016-12-31,1,Ativo Total,True,NaN,8.049450e+11
8,9512,True,True,1,2018-12-31,NaT,2017-12-31,1,Ativo Total,True,NaN,8.315150e+11
9,9512,True,True,1,2019-12-31,NaT,2018-12-31,1,Ativo Total,True,NaN,8.604730e+11


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2450 entries, 0 to 2449
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   CD_CVM         2450 non-null   object        
 1   fs_type        2450 non-null   int8          
 2   is_cons        2450 non-null   bool          
 3   DT_REFER       2450 non-null   datetime64[ns]
 4   DT_INI_EXERC   0 non-null      datetime64[ns]
 5   DT_FIM_EXERC   2450 non-null   datetime64[ns]
 6   ORDEM_EXERC    2450 non-null   int8          
 7   CD_CONTA       2450 non-null   object        
 8   DS_CONTA       2450 non-null   object        
 9   ST_CONTA_FIXA  2450 non-null   bool          
 10  COLUNA_DF      0 non-null      object        
 11  VL_CONTA       2450 non-null   float64       
dtypes: bool(2), datetime64[ns](3), float64(1), int8(2), object(4)
memory usage: 162.8+ KB


In [55]:
df.DT_REFER.unique()

array(['2010-12-31T00:00:00.000000000', '2011-12-31T00:00:00.000000000',
       '2012-12-31T00:00:00.000000000', '2013-12-31T00:00:00.000000000',
       '2014-12-31T00:00:00.000000000', '2015-12-31T00:00:00.000000000',
       '2016-12-31T00:00:00.000000000', '2017-12-31T00:00:00.000000000',
       '2018-12-31T00:00:00.000000000', '2019-12-31T00:00:00.000000000',
       '2020-09-30T00:00:00.000000000', '2020-12-31T00:00:00.000000000',
       '2020-03-31T00:00:00.000000000', '2020-06-30T00:00:00.000000000',
       '2021-09-30T00:00:00.000000000', '2021-03-31T00:00:00.000000000',
       '2021-06-30T00:00:00.000000000'], dtype='datetime64[ns]')

In [57]:
%%time
c_types = df.dtypes
c_types

CPU times: user 236 µs, sys: 0 ns, total: 236 µs
Wall time: 240 µs


CD_CVM       category
fs_type          int8
               ...   
COLUNA_DF    category
VL_CONTA      float64
Length: 12, dtype: object

In [63]:
for column, c_type in zip(df.columns, df.dtypes):
    if (c_type != 'int8') and (c_type != 'bool') and (c_type != 'float64'):
        print(column, c_type)
        df[f'{column}'] = df[f'{column}'].astype('category')

CD_CVM int32
DT_REFER datetime64[ns]
DT_INI_EXERC datetime64[ns]
DT_FIM_EXERC datetime64[ns]
CD_CONTA object
DS_CONTA object
COLUNA_DF object


In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37114 entries, 0 to 37113
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   CD_CVM         37114 non-null  category
 1   fs_type        37114 non-null  int8    
 2   is_cons        37114 non-null  bool    
 3   DT_REFER       37114 non-null  category
 4   DT_INI_EXERC   25026 non-null  category
 5   DT_FIM_EXERC   37114 non-null  category
 6   ORDEM_EXERC    37114 non-null  int8    
 7   CD_CONTA       37114 non-null  category
 8   DS_CONTA       37114 non-null  category
 9   ST_CONTA_FIXA  37114 non-null  bool    
 10  COLUNA_DF      13916 non-null  category
 11  VL_CONTA       37114 non-null  float64 
dtypes: bool(2), category(7), float64(1), int8(2)
memory usage: 803.3 KB
